# Ontario confirmed COVID-19 cases per Region
> "Map and graphs of confirmed COVID-19 cases in Ontario, Canada"

- author: Sophiah Ho https://github.com/anomal
- categories: [ontario, canada, covid-19, province, toronto, waterloo, ottawa, peel, york region, halton, sudbury]
- image: images/ontario_confirmed_map.png
- permalink: /ontario-confirmed-cases-per-region/

In [1]:
#hide
import urllib, json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import HTML

url = 'https://data.ontario.ca/en/api/3/action/datastore_search?resource_id=455fd63b-603d-4608-8216-7d8647f43350&limit=9999999'  
fileobj = urllib.request.urlopen(url)
vals = json.load(fileobj)
df = pd.DataFrame(data=vals["result"]["records"])
df

,_id,Row_ID,Accurate_Episode_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,1,2020-01-22T00:00:00,50s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,2,2,2020-01-21T00:00:00,50s,MALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,3,3,2020-01-24T00:00:00,20s,FEMALE,Travel-Related,Resolved,Middlesex-London Health Unit,50 King Street,London,N6A 5L7,www.healthunit.com,42.981468,-81.254016
3,4,4,2020-02-05T00:00:00,20s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,5,5,2020-02-16T00:00:00,60s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16603,16604,16604,2020-04-29T00:00:00,80s,MALE,Information pending,Not Resolved,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
16604,16605,16605,2020-04-28T00:00:00,80s,FEMALE,Information pending,Not Resolved,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
16605,16606,16606,2020-04-27T00:00:00,20s,FEMALE,Information pending,Not Resolved,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
16606,16607,16607,2020-04-24T00:00:00,40s,MALE,Information pending,Not Resolved,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912


In [2]:
#hide
import dateutil.parser

mindate = dateutil.parser.parse(df["Accurate_Episode_Date"].min()).date()
maxdate = dateutil.parser.parse(df["Accurate_Episode_Date"].max()).date()

In [3]:
#hide_input
HTML(f"Data is available for the period from {str(mindate)} to {str(maxdate)}.")

In [ ]:
#hide
import re

traveldf = df[df["Case_AcquisitionInfo"] == "Travel-Related"] \
.groupby(["Accurate_Episode_Date", "Reporting_PHU", "Reporting_PHU_Latitude", "Reporting_PHU_Longitude"]) \
.count()["Row_ID"] \
.reset_index() \
.rename(columns={"Row_ID" : "Travelled_Count"}) 

countdf = df \
.groupby(["Accurate_Episode_Date", "Reporting_PHU", "Reporting_PHU_Latitude", "Reporting_PHU_Longitude"]) \
.count()["Row_ID"] \
.reset_index() \
.rename(columns={"Row_ID" : "Count"}) 
countdf

joindf = pd.merge(countdf, traveldf, how="left", \
left_on=["Accurate_Episode_Date", "Reporting_PHU", "Reporting_PHU_Latitude", "Reporting_PHU_Longitude"], \
right_on=["Accurate_Episode_Date", "Reporting_PHU", "Reporting_PHU_Latitude", "Reporting_PHU_Longitude"]) \
.fillna(0)
joindf["Datetime"] = joindf.apply(lambda row: dateutil.parser.parse(row["Accurate_Episode_Date"]), axis=1)

phus = countdf.groupby(by=["Reporting_PHU","Reporting_PHU_Latitude","Reporting_PHU_Longitude"], as_index=False) \
    .first()[["Reporting_PHU","Reporting_PHU_Latitude","Reporting_PHU_Longitude"]].to_dict("index")
dates = pd.date_range(mindate, maxdate).tolist()

for date in dates:
    for i in range(len(phus)):
        phu = phus[i]
        phu_name = phu["Reporting_PHU"]
        if joindf[(joindf["Datetime"] == date) & (joindf["Reporting_PHU"] == phu_name)].shape[0] == 0:
            joindf = joindf.append({"Datetime" : date, "Reporting_PHU" : phu_name, \
                          "Reporting_PHU_Latitude" : phu["Reporting_PHU_Latitude"], \
                          "Reporting_PHU_Longitude" : phu["Reporting_PHU_Longitude"], \
                          "Count" : 0, "Travelled_Count" : 0},ignore_index=True)
joindf["Date"] = joindf.apply(lambda row: str(row["Datetime"].date()), axis=1)

aggdf = joindf[["Date", "Reporting_PHU", "Reporting_PHU_Latitude", "Reporting_PHU_Longitude", \
                "Count", "Travelled_Count"]].sort_values(by=["Date","Reporting_PHU"]).reset_index(drop=True)

def cumulativeSum(df, phu, date):
    return df[(df["Reporting_PHU"] == phu) & (df["Date"] <= date)]["Count"].sum()

def cumulativeTravelled(df, phu, date):
    return df[(df["Reporting_PHU"] == phu) & (df["Date"] <= date)]["Travelled_Count"].sum()

def cumulativeNotTravelled(df, phu, date):
    return cumulativeSum(df, phu, date) - cumulativeTravelled(df, phu, date)

def cumulativePercentTravelled(df, phu, date):
    total = cumulativeSum(df, phu, date)
    if total != 0:
        totalTravelled = cumulativeTravelled(df, phu, date)
        return 100 * totalTravelled / total
    else:
        return 0

def getRegion(phu_name):
    return re.sub("(Health|Public|,).+$", "", phu_name)
    
aggdf["Total"] = aggdf.apply(lambda row: cumulativeSum(aggdf, row["Reporting_PHU"], row["Date"]), axis=1) 
aggdf["Total Travelled"] = aggdf.apply(lambda row: cumulativeTravelled(aggdf, row["Reporting_PHU"], row["Date"]), axis=1) 
aggdf["Total Not Travelled"] = aggdf.apply(lambda row: cumulativeNotTravelled(aggdf, row["Reporting_PHU"], row["Date"]), axis=1) 
aggdf["Total Percent Travelled"] = aggdf.apply(lambda row: cumulativePercentTravelled(aggdf, row["Reporting_PHU"], row["Date"]), axis=1)
aggdf["Region"] = aggdf.apply(lambda row: getRegion(row["Reporting_PHU"]), axis=1)
aggdf

In [ ]:
#hide
latestdf = aggdf[aggdf["Date"] == str(maxdate)].sort_values(by=["Count"])
latestdf

In [ ]:
#hide_input

prefix = "Cumulative Confirmed Cases per Ontario Region"
titlelatest = prefix + " up to " + str(maxdate)

latestdfdesc = latestdf.sort_values(by=["Total", "Total Not Travelled"], ascending=[False,False]).reset_index()

barfig = go.Figure(go.Bar(x=latestdfdesc["Region"], y=latestdfdesc["Total Travelled"], name='Travel-Related'))
barfig.add_trace(go.Bar(x=latestdfdesc["Region"], y=latestdfdesc["Total Not Travelled"], name='Other/Unknown'))
barfig.update_layout(barmode='stack', xaxis={'categoryorder':'array'}, title=titlelatest)

In [ ]:
#hide_input
HTML("For the graphs below, the date used is integrated Public Health Information System (iPHIS)'s best estimated date of COVID-19 disease onset, projected into the past based on factors such as symptom onset, test date, or reported date.")

In [ ]:
#hide_input

startdate = "2020-03-01"
fromto = "from " + startdate + " to " + str(maxdate)
agdf = aggdf[aggdf["Date"] >= startdate]
fig = px.scatter_mapbox(agdf, lat="Reporting_PHU_Latitude", lon="Reporting_PHU_Longitude",     
                        color="Total Percent Travelled", 
                        animation_frame="Date",
                        size="Total", hover_name="Reporting_PHU",
                        color_continuous_scale=[ "red","blue"], 
                        size_max=40, zoom=6, 
                        title="Cumulative Cases per Region over Time " + fromto,
                       center=dict(lat=43.6,lon=-79.7))
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
#hide_input
cumline = px.line(agdf, x="Date", y="Total", color="Region", title=prefix + " " + fromto,
        labels={'Region':'Public Health Unit'}
)
cumline.show()

In [ ]:
#hide_input
dailyline = px.line(agdf, x="Date", y="Count", color="Region", title="Daily Confirmed Cases per Ontario Region " + fromto,
       labels={'Region':'Public Health Unit'}
       )
dailyline.show()

In [ ]:
#hide_input
from datetime import date
HTML(f'Data was last retrieved from <a href="https://data.ontario.ca/en/dataset/confirmed-positive-cases-of-covid-19-in-ontario/resource/455fd63b-603d-4608-8216-7d8647f43350" title="Ontario data">Confirmed positive cases of COVID19 in Ontario</a> on {str(date.today())}.')